# Librerías

In [1]:
#!pip install pysqlite3
#!pip install scikit-lear
!pip install numpy==1.23.5
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505214 sha256=48dda8a32575e1d144c7209d109b7f152f579624aa2ed545feaa83e1e7eb644a
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [4]:
# Manejo de Datos
import numpy as np
import pandas as pd
import sqlite3 as sql
import os
import sys
import datetime

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Estadísticas y Pruebas
import scipy.stats as stats
from scipy.stats import gaussian_kde

# Procesamiento de Datos
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Modelado y Algoritmos
from sklearn import neighbors
import joblib
#from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

# Interactividad
from ipywidgets import interact

# Google Colab
from google.colab import drive
from google.colab import files

# Otros
from collections import Counter

In [5]:
from surprise import Reader, Dataset #Libreria para modelos de recomendación

# Conectar con google drive


In [6]:
drive.flush_and_unmount() #Linea en caso de tener que desconectar el drive por algún tipo de falla

Drive not mounted, so nothing to flush and unmount.


In [7]:
#drive.flush_and_unmount()  #Linea en caso de tener que desconectar el drive por algún tipo de falla
drive.mount('/content/drive') #Linea para conectar al drive

Mounted at /content/drive


In [8]:
path="/content/drive/MyDrive/analitica 3/sistemas_recomendacion" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [9]:
import a_funciones as fn #Importar el documento de funciones para hacer uso de estas

# Base de datos

In [13]:
conn = sql.connect('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/data/db_movies3') #Crear la conexión con la base de datos
cur = conn.cursor() #Creacion del cursos para realizar consultas dentro del mismo SQL

In [14]:
# Creación de cursor para  ejecutar consultas en la base de datos
# Visualizar las tablas contenidas en la base de datos
cur.execute("SELECT name FROM sqlite_master where type='table'")
cur.fetchall()

[('df_terminado',),
 ('reco',),
 ('movies_final',),
 ('ratings_final',),
 ('df_final',)]

#Modelos

In [17]:
df_modelos=joblib.load('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_modelos.joblib')   # Carga el DataFrame de modelos desde un archivo .joblib
df_final=joblib.load('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_final.joblib')  # Carga el DataFrame final desde un archivo .joblib

##Sistema de recomendación basado en contenido KNN

In [18]:
#Se reinicia el índice de ambos DataFrames para que esten alineados y no den problemas
df_modelos = df_modelos.reset_index(drop=True)
df_final = df_final.reset_index(drop=True)

In [19]:
#Se crea el modelo KNN con 10 vecinos y usando la distancia euclidiana
model=neighbors.NearestNeighbors(n_neighbors=10,metric='euclidean')
model.fit(df_modelos)
dist,idlist=model.kneighbors(df_modelos) # se calculan las distancias y los indices de los 10 vecinos más cercano

In [20]:
#Convertimos las matrices dist (distancias) e idlist (índices de vecinos) en dataframes de pandas para facilitar su uso posterior
distancias=pd.DataFrame(dist)
id_list=pd.DataFrame(idlist)

**Modelo con película en específico**

In [21]:
movie_list_name = []  # Lista vacía para almacenar títulos recomendados
movie_name='Avengers: Infinity War - Part I (2018)' # Película ejemplo
movie_id = df_final[df_final['title'] == movie_name].index ### extraer el indice de la película
movie_id = movie_id[0] ## si encuentra varios solo guarde uno


# Iterar sobre los 10 vecinos de esa película para evitar titulos duplicados
for newid in idlist[movie_id]:
    title = df_final.loc[newid].title
    if title not in movie_list_name:
        movie_list_name.append(title)

# Mostrar películas mas parecidas nuestro ejemplo específico
movie_list_name

['Avengers: Infinity War - Part I (2018)',
 'Thor: Ragnarok (2017)',
 'Black Panther (2017)',
 'Guardians of the Galaxy 2 (2017)',
 'Doctor Strange (2016)',
 'Guardians of the Galaxy (2014)']

**Modelo general para cualquier película**

In [22]:
# Modelo general como función que toma como entrada el título de las películas
def movie_recomender(movie_name=list(df_final['title'].value_counts().index)):
    movie_list_name = []
    movie_id = df_final[df_final['title'] == movie_name].index ### extraer el indice del libro
    movie_id=movie_id[0] ## si encuentra varios solo guarde uno

# Iterar sobre los 10 vecinos de esa película para evitar titulos duplicados
    for newid in idlist[movie_id]:
        title = df_final.loc[newid].title
        if title not in movie_list_name:
            movie_list_name.append(title)
    return movie_list_name

print(interact(movie_recomender)) # Activamos el selector interactivo para elegir una película y obtener recomendaciones

interactive(children=(Dropdown(description='movie_name', options=('Forrest Gump (1994)', 'Shawshank Redemption…

<function movie_recomender at 0x7ecffe252ac0>


##Sistema de recomendación basado en contenido KNN (Basado en el usuario)

In [23]:
## seleccionar usuarios para recomendaciones
usuarios=pd.read_sql('select distinct (user_id) as user_id from ratings_final',conn)

In [24]:
# Función para recomendar películas a un usuario basado en su perfil de gustos usando k-vecinos más cercanos
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id})

    ###convertir ratings del usuario a array
    l_movies_r=ratings['movie_id'].to_numpy()

    ###agregar la columna de movie_id y titulo de la pelicula a dummie para filtrar y mostrar nombre
    df_modelos[['movie_id','title']]=df_final[['movie_id','title']]

    ### filtrar las peliculas calificadas por el usuario
    movies_r=df_modelos[df_modelos['movie_id'].isin(l_movies_r)]

    ## eliminar columna nombre e movie_id
    movies_r=movies_r.drop(columns=['movie_id','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean()


    ### filtrar peliculas no vistas
    books_nr=df_modelos[~df_modelos['movie_id'].isin(l_movies_r)]
    ## eliminbar nombre y movie_id
    books_nr=books_nr.drop(columns=['movie_id','title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(books_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=df_final.loc[ids][['title','movie_id']]
    leidos=df_final[df_final['movie_id'].isin(l_movies_r)][['title','movie_id']]

    return recomend_b


recomendar(550) #Ejecutar la funcion con user_id (usuario) de prueba

,title,movie_id
76587,For Your Eyes Only (1981),2989
63521,Mission: Impossible (1996),648
56042,Lost in Translation (2003),6711
56027,Bruce Almighty (2003),6373
40318,"Hobbit: An Unexpected Journey, The (2012)",98809
21536,Jurassic Park (1993),480
21520,Bullets Over Broadway (1994),348
63502,Pulp Fiction (1994),296
29611,Fear and Loathing in Las Vegas (1998),1884
15189,"Wolf of Wall Street, The (2013)",106782


In [25]:
print(interact(recomendar)) # Crea un widget interactivo para seleccionar el usuario y mostrar recomendaciones en tiempo real

interactive(children=(Dropdown(description='user_id', options=(1, 410, 403, 404, 405, 406, 407, 408, 409, 411,…

<function recomendar at 0x7ecffe253880>


##Sistema de recomendación filtro colaborativo

In [26]:
# Cargar toda la tabla 'ratings_final'
ratings=pd.read_sql('select * from ratings_final', conn)
ratings.head()

,user_id,movie_id,rating,year_ratings,month,day
0,1,1,4.0,2000,7,30
1,1,3,4.0,2000,7,30
2,1,6,4.0,2000,7,30
3,1,47,5.0,2000,7,30
4,1,50,5.0,2000,7,30


In [27]:
reader = Reader(rating_scale=(1, 10)) # Define el rango de los ratings para el lector de datos de Surprise
data   = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader) # Crea un conjunto de datos para Surprise a partir del dataframe de ratings

In [28]:
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] # Lista de modelos KNN de Surprise para probar diferentes variantes
results = {} # Diccionario para almacenar resultados de los modelos

In [ ]:
model=models[1] # Elección de un modelo incial para la iteración
for model in models: # Iterar sobre cada modelo de la lista de modelos

    # Realizar validación cruzada (5 subdivisiones) calculando MAE y RMSE
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    # Calculo del promedio de las metricas y guardar los resultados en el diccionario
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T # Convertir diccionario de resultados en un dataframe
performance_df.sort_values(by='RMSE') # Ordenar los modelos en funcion de su RMSE de menor a mayor

En los resultados obtenidos, el modelo KNNBaseline presenta el mejor desempeño, con el MAE más bajo (0.6429) y también el menor RMSE (0.8408), lo que indica una mayor precisión en la predicción de las calificaciones de los usuarios. Aunque no es el modelo más rápido en términos de tiempo de entrenamiento y prueba, la diferencia respecto a los demás es mínima y no representa un factor determinante.

In [ ]:
# Diccionario con parametros que se probaran durante la busqueda de hiperparámetros
# donde, 'msd' es (Mean Squared Difference), 'cosine' es (similitud coseno) y min_support es
# número mínimo de coincidencias (usuarios o ítems comunes) para considerar la similitud
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}
             }

In [ ]:
# Creamos un objeto GridSearchCV para realizar la búsqueda de hiperparámetros
gridsearchKNNBaseline = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1) # cv: número de particiones para la validación cruzada (2 en este caso)
                                      # n_jobs=-1 usa todos los núcleos disponibles, es decir numero de trabajos en paralelo

gridsearchKNNBaseline.fit(data) # Entrenamiento del modelo

In [ ]:
# Obtener el mejor conjunto de hiperparámetros encontrados durante la búsqueda, especificamente los que lograron un mejor RMSE
gridsearchKNNBaseline.best_params["rmse"]
gridsearchKNNBaseline.best_score["rmse"]
gs_model=gridsearchKNNBaseline.best_estimator['rmse']

In [ ]:
trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
#joblib.dump(model,'/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/recommodel.joblib')

In [ ]:
### Hacer predicción con el modelo de recomendación
# uid: ID del usuario para quien se quiere predecir la calificación
# iid: ID del ítem (película) para el cual se quiere hacer la predicción
model.predict(uid=5, iid='5',r_ui='')

Prediction(uid=5, iid='5', r_ui='', est=3.5093755789717713, details={'was_impossible': False})

In [ ]:
#Crea el anti-testset: una lista de todas las combinaciones (usuario, película) que NO existen en el conjunto de entrenamiento
predset = trainset.build_anti_testset()
len(predset)

#Muestra las primeras 10 combinaciones usuario-película no vistas,
predset[0:10]

[(1, 318, 3.579442714350294),
 (1, 1704, 3.579442714350294),
 (1, 6874, 3.579442714350294),
 (1, 8798, 3.579442714350294),
 (1, 46970, 3.579442714350294),
 (1, 48516, 3.579442714350294),
 (1, 58559, 3.579442714350294),
 (1, 60756, 3.579442714350294),
 (1, 68157, 3.579442714350294),
 (1, 71535, 3.579442714350294)]

In [ ]:
# Utiliza el modelo entrenado (gs_model) para predecir las calificaciones
# de todas las combinaciones usuario-película que no han sido calificadas
predictions = gs_model.test(predset)

In [ ]:
#Muestra las primeras 10 predicciones
predictions[0:10]

[Prediction(uid=1, iid=318, r_ui=3.579442714350294, est=4.953888892997183, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=1704, r_ui=3.579442714350294, est=4.811858760280403, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=6874, r_ui=3.579442714350294, est=4.745184479485489, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=8798, r_ui=3.579442714350294, est=4.6102450867763745, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=46970, r_ui=3.579442714350294, est=4.139481333486526, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=48516, r_ui=3.579442714350294, est=4.9620836959599846, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=58559, r_ui=3.579442714350294, est=5.009553009135706, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1, iid=60756, r_ui=3.579442714350294, est=4.232274503384751, details={'actual_k': 40

In [ ]:
predictions_df = pd.DataFrame(predictions) #Convertir la lista de predicciones en un dataframe
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df. sort_values(by='est',ascending=False) # Ordenar el dataframe enforma descendente respecto a est ---- est: Es la calificación predicha

,uid,iid,r_ui,est,details
103970,53,318,3.579443,5.745741,"{'actual_k': 20, 'was_impossible': False}"
104487,53,140110,3.579443,5.668647,"{'actual_k': 1, 'was_impossible': False}"
104310,53,750,3.579443,5.637350,"{'actual_k': 19, 'was_impossible': False}"
103932,53,2959,3.579443,5.630968,"{'actual_k': 20, 'was_impossible': False}"
104046,53,904,3.579443,5.593950,"{'actual_k': 20, 'was_impossible': False}"
...,...,...,...,...,...
5477,3,3451,3.579443,1.000000,"{'actual_k': 15, 'was_impossible': False}"
880430,442,158,3.579443,1.000000,"{'actual_k': 20, 'was_impossible': False}"
4326,3,2770,3.579443,1.000000,"{'actual_k': 24, 'was_impossible': False}"
4114,3,2054,3.579443,1.000000,"{'actual_k': 24, 'was_impossible': False}"


In [ ]:
#Llevar dataframe de predicciones de peliculas a un csv
#predictions_df.to_csv('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/predictions_movies.csv', index=False)

In [ ]:
#Funcion que genera recomendaciones personalizadas de películas para un usuario específico basadas en
#predicciones de ratings, y además incluye los títulos de las películas

def recomendaciones(user_id, n_recomend=10):

    # Filtrar las predicciones para el usuario y ordenar por la calificación estimada
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                        sort_values(by="est", ascending=False).head(n_recomend)

    # Seleccionar las columnas necesarias y renombrarlas
    recomendados = predictions_userID[['uid', 'iid', 'r_ui', 'est']]
    recomendados.columns = ['user_id', 'movie_id', 'promedio_rating_real', 'estimacion_rating']

    # Guardar las recomendaciones en la base de datos
    recomendados.to_sql('reco', conn, if_exists="replace", index=False)

    # Realizar la consulta SQL para obtener los títulos de las películas y eliminar duplicados
    recomendados = pd.read_sql('''SELECT a.*, b.title
                                  FROM reco a
                                  LEFT JOIN df_final b
                                  ON a.movie_id = b.movie_id''', conn)

    # Eliminar filas duplicadas
    recomendados = recomendados.drop_duplicates(subset=['movie_id', 'title'])

    return recomendados

# Ejemplo
recomendaciones(user_id=100, n_recomend=600)

,user_id,movie_id,promedio_rating_real,estimacion_rating,title
0,100,3147,3.579443,4.407402,"Green Mile, The (1999)"
111,100,318,3.579443,4.400224,"Shawshank Redemption, The (1994)"
428,100,58559,3.579443,4.399551,"Dark Knight, The (2008)"
577,100,2324,3.579443,4.392307,Life Is Beautiful (La Vita è bella) (1997)
665,100,48516,3.579443,4.377912,"Departed, The (2006)"
...,...,...,...,...,...
30379,100,79702,3.579443,4.000788,Scott Pilgrim vs. the World (2010)
30423,100,6957,3.579443,4.000457,Bad Santa (2003)
30449,100,112556,3.579443,4.000365,Gone Girl (2014)
30486,100,262,3.579443,4.000307,"Little Princess, A (1995)"
